In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 53.6 MB/s 
     |████████████████████████████████| 7.6 MB 27.9 MB/s 


In [3]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score



In [5]:
from inference import QAModelInference
from preprocess import SquadPlausibleAnswersPreprocessor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def evaluate_possible(correct_ans, predicted_ans):
    ans = predicted_ans['answer'].replace(" ##", "").replace("  ,", ",").replace(" ,", ",")

    if correct_ans['text'].lower() == ans:
        return 1
    return 0

In [ ]:
possible_model_url= ""https://www.dropbox.com/s/fgyyokik58wsvha/model_possible.pt?dl=1""

In [7]:
inf = QAModelInference(models_path="/content/drive/MyDrive/models", plausible_model_fn="/content/drive/MyDrive/models/plausible_model_500.pt",
                       possible_model_fn="/content/drive/MyDrive/model_checkpoint/model_possible.pt")


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- T

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
sp = SquadPlausibleAnswersPreprocessor()

contexts, questions, answers, is_impossible = sp._read_squad(path = "/content/drive/MyDrive/dev-v2.0.json",
    frac=1.0,
    include_impossible=True)

predicted_impossible, is_correct = [], []
n = 300

In [9]:
for context, question,answer in tqdm(zip(contexts[:n], questions[:n], answers[:n])):
    ans = inf.extract_answer(context, question)
    if ans['plausible_answer'] :
        predicted_impossible.append(1)
    else:
        proba_po = (np.max(ans['start_word_proba_possible_model'])+np.max(ans['end_word_proba_possible_model']))/2
        #proba_pl = (np.max(ans['start_word_proba_plausible_model'])+np.max(ans['end_word_proba_plausible_model']))/2
        proba = 1-proba_po


        predicted_impossible.append(proba)
    is_correct.append(evaluate_possible(answer, ans))


12it [00:11,  1.19it/s]WARNING:root:Error: start_idx = 49, end_idx = 97
13it [00:11,  1.25it/s]WARNING:root:Error: start_idx = 49, end_idx = 97
14it [00:12,  1.30it/s]WARNING:root:Error: start_idx = 49, end_idx = 97
15it [00:13,  1.34it/s]WARNING:root:Error: start_idx = 49, end_idx = 97
21it [00:17,  1.37it/s]WARNING:root:Error: start_idx = 49, end_idx = 97
34it [00:31,  1.13s/it]WARNING:root:Error: start_idx = 276, end_idx = 4
38it [00:35,  1.14s/it]WARNING:root:Error: start_idx = 4, end_idx = 78
39it [00:36,  1.07it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
40it [00:36,  1.24it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
41it [00:37,  1.41it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
42it [00:37,  1.57it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
43it [00:38,  1.69it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
44it [00:38,  1.76it/s]WARNING:root:Error: start_idx = 4, end_idx = 78
45it [00:39,  1.87it/s]WARNING:root:Error: start_idx = 4, end_idx = 78


In [10]:
print(predicted_impossible)
print(is_impossible[:n])

print("How well model detects impossible questions: ")
print(roc_auc_score(is_impossible[:n], predicted_impossible))
print(average_precision_score(is_impossible[:n], predicted_impossible))

is_correct = np.array(is_correct)

select = [not bool(x) for x in is_impossible[:n]]

print("Accuracy overall: ", np.mean(is_correct))
print("Accuracy (only possible): ", np.mean(is_correct[select]))

[1, 1, 1, 1, 0.18512725830078125, 0.18512725830078125, 0.18512725830078125, 0.18512725830078125, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5698200762271881, 1, 1, 1, 1, 1, 1, 1, 1, 0.39463192224502563, 0.39463192224502563, 0.39463192224502563, 1, 1, 0.3829802870750427, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.45506006479263306, 0.45506006479263306, 0.45506006479263306, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.4679850935935974, 0.4679850935935974, 0.4679850935935974, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,